In [1]:
import numpy as np
import imutils
import cv2
from matplotlib import pyplot as plt
from scipy.linalg import null_space

ratio = 0.75
reprojThresh = 4.0

In [2]:
# Amount of images
# Their names should be 1.jpg, 2.jpg, ...
N = 3

In [3]:
def detectAndDescribe(image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        descriptor = cv2.xfeatures2d.SIFT_create()
        (kps, features) = descriptor.detectAndCompute(image, None)
        kps = np.float32([kp.pt for kp in kps])
        return (kps, features)
    
def matchKeypoints(kps1, kps2, featuresA, featuresB, ratio, reprojThresh):
        matcher = cv2.DescriptorMatcher_create("BruteForce")
        rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
        pts1_ = []
        pts2_ = []
        for m in rawMatches:
            if len(m) == 2 and m[0].distance < m[1].distance * ratio:
                pts1_.append(m[0].trainIdx)
                pts2_.append(m[0].queryIdx)
        pts1 = np.float32([kps1[i] for i in pts2_])
        pts2 = np.float32([kps2[i] for i in pts1_])
        return (pts1, pts2)
    
def get_homography(pts1, pts2):
    X = []
    for i in range(4):
        X.append([-pts1[i][0], -pts1[i][1], -1, 0, 0, 0, pts2[i][0]*pts1[i][0], pts2[i][0]*pts1[i][1], pts2[i][0]])
        X.append([0, 0, 0, -pts1[i][0], -pts1[i][1], -1, pts2[i][1]*pts1[i][0], pts2[i][1]*pts1[i][1], pts2[i][1]])
    H = null_space(X)
    H = H.T
    H = H[0]
    H = H / H[8]
    H = np.array(H)
    H = H.reshape((3, 3))
    H = np.linalg.inv(H)
    H = H / H[2][2]
    return H

def mod(x1, x2):
    return ((x1[0] - x2[0])**2 + (x1[1] - x2[1])**2 + (x1[2] - x2[2])**2)**0.5

def E(H, pts1, pts2, epsilon=10):
    result = 0
    for i in range(len(pts1)):
        result += int(mod(np.concatenate((pts1[i], [1])),
                          H.dot(np.concatenate((pts2[i], [1])))) > epsilon)
    return result

def pick_random_4(pts1, pts2):
    rand_perm = np.random.permutation(len(pts1))
    pts1_ = []
    pts2_ = []
    for i in range(4):
        pts1_.append(pts1[rand_perm[i]])
        pts2_.append(pts2[rand_perm[i]])
    return pts1_, pts2_

def RANSAC(pts1, pts2, epsilon=10):
    H_best = np.zeros((3, 3), np.float32)
    E_best = 1000000
    iterations = 1000
    while iterations > 0:
        pts1_, pts2_ = pick_random_4(pts1, pts2)
        H_cur = get_homography(pts1_, pts2_)
        E_cur = E(H_cur, pts1, pts2, epsilon)
        if E_cur < E_best:
            E_best = E_cur
            H_best = H_cur
        iterations -= 1
    return H_best
    
def draw(img, pts):
    _, c, _ = img.shape
    for pt in pts:
        color = tuple(np.random.randint(0, 255, 3).tolist())
        img = cv2.circle(img, tuple(pt), 5, color, -1)
    return img

def get_src_points(H, y, x):
    y = H[0][0] * float(y) + H[0][1] * float(x) + H[0][2]
    x = H[1][0] * float(y) + H[1][1] * float(x) + H[1][2]
    z = H[2][0] * float(y) + H[2][1] * float(x) + H[2][2]
    return y / z, x / z

def panorama(H, img1, img2):
    h, w, c = img1.shape
    img = np.zeros([3*h, 3*w, c], np.uint8)
    for y in range(3*h):
        for x in range(3*w):
            vec = H.dot([x - w, y - h, 1])
            vec /= vec[2]
            y_new = vec[1]
            x_new = vec[0]
            if y_new >= 0 and y_new < h - 1 and x_new >= 0 and x_new < w - 1:
                y0 = int(y_new)
                y1 = int(y_new + 0.5)
                x0 = int(x_new)
                x1 = int(x_new + 0.5)
                a0 = a1 = a2 = a3 = 0.25
                if y1 == y0:
                    a0 = 0.5
                    a1 = 0.5
                    a2 = 0
                    a3 = 0
                if x1 == x0:
                    a0 += a1
                    a1 = 0
                    a2 += a3
                    a3 = 0
                if img[y][x][0] == 0 and img[y][x][1] == 0 and img[y][x][2] == 0:
                    img[y][x] = a0 * img1[y0][x0] + a1 * img1[y0][x1] + a2 * img1[y1][x0] + a3 * img1[y1][x1]
                else:
                    img[y][x] = (img[y][x] + a0 * img1[y0][x0] + a1 * img1[y0][x1] + a2 * img1[y1][x0] + a3 * img1[y1][x1]) / 2
                
    return img

def get_pic(img1, img2, kps2, features2):
    h, w, _ = img2.shape
    (kps1, features1) = detectAndDescribe(img1)
    (pts1, pts2) = matchKeypoints(kps1, kps2, features1, features2, ratio, reprojThresh)
    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    H = RANSAC(pts1, pts2, 10)
    img = panorama(H, img1, img2)
    return img

In [4]:
img1 = cv2.imread("1.jpg")
img1 = imutils.resize(img1, width=500)

(kps1, features1) = detectAndDescribe(img1)

h, w, c = img1.shape
img = np.zeros([3*h, 3*w, c], np.uint8)
img[h:2*h, w:2*w] = img1

cv2.imwrite("res1.png", img)

for i in range(2, N + 1):
    img2 = cv2.imread(str(i) + ".jpg")
    img2 = imutils.resize(img2, width=500)
    img = get_pic(img2, img1, kps1, features1)
    cv2.imwrite("res" + str(i) + ".png", img)